In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
import re
import shutil
import time
from datetime import datetime
from datetime import timedelta
pd.set_option('display.max_columns',None)
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [4]:
'''更换数据位置路径'''
os.chdir('C:\\Users\\Administrator\\Desktop\\重大风险事项客户数据\\datasource')
path=os.getcwd()

In [29]:
'''read debt cust data'''
debt_cust=pd.read_excel('客户债项信息.xlsx',parse_dates=True)

In [26]:
'''对数据groupby并sort'''
def data_groupby_sort(column_name):
    data=debt_cust.groupby([column_name]).count()
    data=data.loc[:,['客户名称','客户编号']]
    data=data.reset_index().loc[:,[column_name,'客户编号']]
    data=data.sort_values(by=['客户编号'],ascending=[False])
    data=data.set_index([column_name])
    return data

In [116]:
debt_cust_branch=data_groupby_sort('客户管辖一级行名称')  # group by branch
debt_cust_industry=data_groupby_sort('所属行业门类名称')  # group by industry
debt_cust_sub_industry=data_groupby_sort('所属行业大类名称') # group by sub indusrty
display('debt_cust_branch','debt_cust_industry','debt_cust_sub_industry.iloc[:10,:]')

,客户编号
客户管辖一级行名称,
建行浙江省分行,704
建行山东省分行,473
建行江苏省分行,271
建行福建省分行,258
建行广东省分行,213
建行宁波市分行,174
建行内蒙古自治区分行,115
建行湖北省分行,100
建行吉林省分行,88


In [107]:
'''read debt cust amount data'''
debt_cust_amount=pd.read_excel('客户10年债项余额情况统计.xlsx',parse_dates=True)


In [109]:
'''modify debt cust amount data '''
def debt_decade_variation(column_name):
    # pivottable  fillna div100000000
    debt_cust_amount_pivot=debt_cust_amount.pivot_table('信贷余额',index=['客户编号'],columns=['报告期']).fillna(0)/100000000 
    debt_cust_amount_pivot=debt_cust_amount_pivot.reset_index()
    
    # get cust information from debt_cust 
    debt_cust_inf=debt_cust.loc[:,['客户编号',column_name]]
    
    # inner join cust amount and cust information
    debt_cust_amount_inf=pd.merge(debt_cust_amount_pivot,debt_cust_inf,on='客户编号',how='inner')
    
    # groupby and merge cust amount data
    trail_data_orginal=debt_cust_amount_inf.groupby([column_name]).aggregate({datetime(2020,6,30):'sum'})
    for i  in pd.date_range('2010-12-31','2019-12-31',freq='Y'):
        trail_data=debt_cust_amount_inf.groupby([column_name]).aggregate({i:'sum'})
        trail_data_orginal=pd.merge(trail_data_orginal,trail_data,on=column_name,how='left')
        
    # sort columns 
    trail_data_orginal=trail_data_orginal.reindex(columns=trail_data_orginal.columns.sort_values(ascending=True))
    
    return trail_data_orginal

In [115]:
debt_decade_variation('所属行业大类名称').reindex(debt_cust_sub_industry.index)
debt_decade_variation('客户管辖一级行名称').reindex(debt_cust_branch.index).head(1)

,2010-12-31 00:00:00,2011-12-31 00:00:00,2012-12-31 00:00:00,2013-12-31 00:00:00,2014-12-31 00:00:00,2015-12-31 00:00:00,2016-12-31 00:00:00,2017-12-31 00:00:00,2018-12-31 00:00:00,2019-12-31 00:00:00,2020-06-30 00:00:00
客户管辖一级行名称,,,,,,,,,,,
建行浙江省分行,684.817321,785.198744,777.220511,717.274519,615.509799,387.796548,274.197025,196.49659,132.15196,95.11388,75.170688


In [148]:
'''read approval cust amount data'''
approval_cust_amount=pd.read_excel('客户审批信息.xlsx',parse_dates=True)
approval_cust_amount['申报授信/明细支用金额（合人民币）']= \
approval_cust_amount['申报授信/明细支用金额（合人民币）']/100000000
approval_cust_amount['批复授信/明细支用金额（合人民币）']= \
approval_cust_amount['批复授信/明细支用金额（合人民币）']/100000000

In [155]:
'''general statistic '''
approval_cust_amount.groupby(['审批判断']).aggregate({'业务编号':'count',
                                                                      '申报授信/明细支用金额（合人民币）':'sum',
                                                                      '批复授信/明细支用金额（合人民币）':'sum',})

,业务编号,申报授信/明细支用金额（合人民币）,批复授信/明细支用金额（合人民币）
审批判断,,,
减额,2127,6212.106908,5035.392911
变更授信方案项目,2295,6769.146879,6740.205600
同意,13983,27331.451192,27410.380155
否决,175,503.310793,0.000000
续议,324,773.071864,0.000000


In [150]:
'''modify approval cust amount data '''
def approval_groupby(column_name):
    debt_cust_inf=debt_cust.loc[:,['客户编号',column_name]]
    approval_cust_amount_inf=pd.merge(approval_cust_amount,debt_cust_inf,on='客户编号',how='inner')
    approval_cust_amount_groupby=approval_cust_amount_inf.groupby([column_name,'审批判断']).aggregate({'业务编号':'count',
                                                                      '申报授信/明细支用金额（合人民币）':'sum',
                                                                      '批复授信/明细支用金额（合人民币）':'sum',})
    return approval_cust_amount_groupby
    

In [153]:
approval_groupby('所属行业大类名称').head(2)
approval_groupby('客户管辖一级行名称').head(2)

业务编号  申报授信/明细支用金额（合人民币）  批复授信/明细支用金额（合人民币）
客户管辖一级行名称 审批判断                                                
建行上海市分行   减额          16            87.0671          57.621400
          变更授信方案项目    32           128.7183         126.284723

In [156]:
'''read financial data'''
financial_cust_amount=pd.read_excel('客户财务数据.xlsx',parse_dates=True)
risk_cust_inf=pd.read_excel('客户名单.xlsx','查询客户名单',parse_dates=True)

In [161]:
financial_cust_amount.dtypes

客户编号                      int64
客户名称                     object
财务报表排序(从近往远)              int64
财务报表截止日期         datetime64[ns]
是否经过审计                   object
财务报表类型                   object
财务报表口径                   object
财务报表币种                   object
货币资金                    float64
应收票据                    float64
其它应收款                   float64
存货                      float64
长期投资                    float64
固定资产净值                  float64
短期借款                    float64
应付票据                    float64
应付账款                    float64
预收账款                    float64
其它应付款                   float64
资产合计                    float64
负债合计                    float64
所有者权益合计                 float64
产品销售收入                  float64
产品销售成本                  float64
产品销售费用                  float64
产品销售利润                  float64
利润总额                    float64
净利润                     float64
经营活动产生的现金流量净额           float64
投资活动产生的现金流量净额           float64
筹资活动产生的现金流量净额           float64
存货周转率   

In [166]:
'''select  prior_risk financial report '''
financial_cust_amount['财务报表年份']=financial_cust_amount['财务报表截止日期'].dt.year
financial_cust_amount_inf=pd.merge(financial_cust_amount,risk_cust_inf,on='客户名称',how='inner')
financial_cust_amount_inf['财务时间日期判断']=financial_cust_amount_inf['年份']-financial_cust_amount_inf['财务报表年份']
financial_cust_amount_inf=financial_cust_amount_inf.loc[financial_cust_amount_inf['财务时间日期判断']>=0,:]

financial_cust_amount_inf['sort_id']= \
financial_cust_amount_inf['财务报表截止日期'].groupby(financial_cust_amount_inf['客户名称']) \
.rank(method='dense',ascending=True)

In [170]:
financial_cust_amount_inf.sort_values(by=['客户名称','sort_id'],ascending=[True,True])
financial_cust_amount_inf['sort_id'].max()

9.0